In [79]:
import pandas as pd
import numpy as np

# Load raw dataset
df = pd.read_csv('/workspaces/ML_01_video_game_prediction/video_game_analysis/src/data/Video_Games_Sales_as_at_22_Dec_2016.csv')

# Preview first few rows
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#Checking for missing values
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [81]:
#Drop rows where the target or essential info is missing
#Since global Sales null values are 0, didnt include
df = df.dropna(subset=['Year_of_Release']) 

# Convert year to integer
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

In [82]:
# Convert 'tbd' to NaN and then to float
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

In [83]:
# Fill numeric scores & counts with median (fixed version)
median_values = {
    'Critic_Score': df['Critic_Score'].median(),
    'Critic_Count': df['Critic_Count'].median(),
    'User_Score': df['User_Score'].median(),
    'User_Count': df['User_Count'].median()
}
df.fillna(median_values, inplace=True)

# Fill categories with 'Unknown' (fixed version)
df['Publisher'] = df['Publisher'].fillna('Unknown')
df['Developer'] = df['Developer'].fillna('Unknown')
df['Rating'] = df['Rating'].fillna('Unknown')


In [84]:
#Feature Engineering
# User_Score is on a 10 scale, so convert it to 100 before averaging
df['Total_Rating'] = (df['Critic_Score'] + (df['User_Score'] * 10)) / 2

In [85]:
#Drop Unnecessary Columns
df.drop(columns=['Name', 'Developer'], inplace=True)

In [86]:
#Handle Categorical Variables
top_publishers = df['Publisher'].value_counts().nlargest(30).index
df['Publisher'] = df['Publisher'].apply(lambda x: x if x in top_publishers else 'Other')

#One-hot encode categorical columns
df = pd.get_dummies(df, columns=['Platform', 'Genre', 'Publisher', 'Rating'], drop_first=True)

In [87]:
#Confirm Everything is Numeric
df.info()  # Make sure all columns are now numeric

<class 'pandas.core.frame.DataFrame'>
Index: 16450 entries, 0 to 16718
Data columns (total 90 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Year_of_Release                                   16450 non-null  int64  
 1   NA_Sales                                          16450 non-null  float64
 2   EU_Sales                                          16450 non-null  float64
 3   JP_Sales                                          16450 non-null  float64
 4   Other_Sales                                       16450 non-null  float64
 5   Global_Sales                                      16450 non-null  float64
 6   Critic_Score                                      16450 non-null  float64
 7   Critic_Count                                      16450 non-null  float64
 8   User_Score                                        16450 non-null  float64
 9   User_Count            

In [88]:
#Save Cleaned Dataset
df.to_csv('/workspaces/ML_01_video_game_prediction/video_game_analysis/notebooks/FC110557_Siyas/vgsales_cleaned.csv', index=False)